In [45]:
import torch
torch.cuda.is_available()

True

In [100]:
import pandas as pd 
data = pd.read_csv("C:/Study/Master Thesis/expaai_model/tmp/cleaned_data_150k.csv") 
data.columns = ['text', 'label']

# data.head()

data.to_csv('../tmp/cleaned_data_with_labels_150k.csv', index=False)

In [101]:
from datasets import load_dataset
paragraph_data = load_dataset("csv", data_files="C:/Study/Master Thesis/expaai_model/tmp/cleaned_data_with_labels_150k.csv")
# paragraph_data["train"][0]

Using custom data configuration default-d6609920110419df


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\ranax\.cache\huggingface\datasets\csv\default-d6609920110419df\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [166]:
paragraph_data_randomized = paragraph_data["train"].shuffle(seed=42)
paragraph_data_randomized_rows = paragraph_data.num_rows['train']
train_end = round(paragraph_data_randomized_rows * 0.67)

paragraph_train_dataset = paragraph_data_randomized.select([i for i in list(range(train_end))])
paragraph_test_dataset = paragraph_data_randomized.select([i for i in list(range(train_end, paragraph_data_randomized_rows))])

#paragraph_train_dataset
#paragraph_test_dataset

Loading cached shuffled indices for dataset at C:\Users\ranax\.cache\huggingface\datasets\csv\default-d6609920110419df\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-7619a88dcc424e83.arrow


In [167]:
len(paragraph_train_dataset[0]['text'])

203

In [168]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("anferico/bert-for-patents")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/anferico/bert-for-patents/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\ad9665ebfb3752215321190b40835afc46da469265b8bd17501173eb6f7ad8c0.607ba830f1d59bcf5505774afff2deee461950e07e7e50c9e3d4595f91401ba0
Model config BertConfig {
  "_name_or_path": "anferico/bert-for-patents",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vo

In [169]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True,max_length=50, add_special_tokens = True)

tokenized_train = paragraph_train_dataset.map(preprocess_function, batched=True)
tokenized_test = paragraph_test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\ranax\.cache\huggingface\datasets\csv\default-d6609920110419df\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a\cache-adb5e3442e064574.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

In [170]:
tokenized_test[0]

{'text': 'According to the organic EL display panel and the method for manufacturing the same of the present invention can make a film shape of the organic functional layer of the light emitting element at an edge of the effective emission region favorable. In addition, it is possible to reduce the number of the contact holes and the holes formed in the planarized film and to thereby make the surface area of the planarized film as small as possible. Therefore, it is possible to further suppress absorption of moisture in the planarized film and diffusion of moisture from the planarized film into other layers. For this reason, the organic EL display panel of the present invention can achieve small luminance unevenness and emission color unevenness and high display quality, and can further suppress deterioration in emission characteristics.',
 'label': 1,
 'input_ids': [2,
  2094,
  1665,
  1661,
  7219,
  3114,
  4318,
  5662,
  1663,
  1661,
  3783,
  1670,
  5479,
  1661,
  1833,
  166

In [171]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [172]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("anferico/bert-for-patents", num_labels=3)

loading configuration file https://huggingface.co/anferico/bert-for-patents/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\ad9665ebfb3752215321190b40835afc46da469265b8bd17501173eb6f7ad8c0.607ba830f1d59bcf5505774afff2deee461950e07e7e50c9e3d4595f91401ba0
Model config BertConfig {
  "_name_or_path": "anferico/bert-for-patents",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version":

In [161]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="micro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [162]:
# load_accuracy = load_metric("accuracy")
# load_accuracy.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])

load_f1 = load_metric("f1")
load_f1.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0], average="micro")

# from datasets import list_metrics
# list_metrics()

{'f1': 0.5}

In [75]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\ranax/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [173]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "bert-for-patent"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
c:\Study\Master Thesis\expaai_model\BertPatent\bert-for-patent is already a clone of https://huggingface.co/fassahat/bert-for-patent. Make sure you pull the latest changes with `repo.git_pull()`.


In [174]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 49500
  Batch size = 16


  0%|          | 0/3094 [00:00<?, ?it/s]

{'eval_loss': 1.2239919900894165,
 'eval_accuracy': 0.3336767676767677,
 'eval_f1': 0.3336767676767677,
 'eval_runtime': 1847.1688,
 'eval_samples_per_second': 26.798,
 'eval_steps_per_second': 1.675}

In [93]:
trainer.push_to_hub()

Saving model checkpoint to bert-for-patent
Configuration saved in bert-for-patent\config.json
Model weights saved in bert-for-patent\pytorch_model.bin
tokenizer config file saved in bert-for-patent\tokenizer_config.json
Special tokens file saved in bert-for-patent\special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/1.28G [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.23k/3.23k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/fassahat/bert-for-patent
   12a147c..5e7a0fa  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}}
To https://huggingface.co/fassahat/bert-for-patent
   5e7a0fa..c17c321  main -> main



'https://huggingface.co/fassahat/bert-for-patent/commit/5e7a0fa3c1c39f9745995132e92beef18bcfa05f'

In [94]:
from transformers import pipeline
 
bert_patent_model = pipeline(model="fassahat/bert-for-patent")
bert_patent_model(["', 'The vibrator may be pressed with a pressing force in the range from 3N to 10N."])

https://huggingface.co/fassahat/bert-for-patent/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\ranax\.cache\huggingface\transformers\tmpjm1f2o0f


storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/config.json in cache at C:\Users\ranax/.cache\huggingface\transformers\805662ad68bc5873735c511c6d9ae90801f2bc14d9c22acf62ec5dd96eecc962.c530123a1af49ed0d4a7d7d6ec7c7cc0e9e385346735a75aa51aa6768a2bd9ae
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\805662ad68bc5873735c511c6d9ae90801f2bc14d9c22acf62ec5dd96eecc962.c530123a1af49ed0d4a7d7d6ec7c7cc0e9e385346735a75aa51aa6768a2bd9ae
loading configuration file https://huggingface.co/fassahat/bert-for-patent/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\805662ad68bc5873735c511c6d9ae90801f2bc14d9c22acf62ec5dd96eecc962.c530123a1af49ed0d4a7d7d6ec7c7cc0e9e385346735a75aa51aa6768a2bd9ae
Model config BertConfig {
  "_name_or_path": "fassahat/bert-for-patent",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidd

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

loading configuration file https://huggingface.co/fassahat/bert-for-patent/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\805662ad68bc5873735c511c6d9ae90801f2bc14d9c22acf62ec5dd96eecc962.c530123a1af49ed0d4a7d7d6ec7c7cc0e9e385346735a75aa51aa6768a2bd9ae
Model config BertConfig {
  "_name_or_path": "fassahat/bert-for-patent",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtyp

storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/pytorch_model.bin in cache at C:\Users\ranax/.cache\huggingface\transformers\4a9afff6de7545db1ef56fb4ecace0a47cc44705dda20f10c0bd64f509fea7cd.0ef7cb179a37032843c28b2c55b8ba7d312d6e44596aa433a59a40fc8abb8f72
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\4a9afff6de7545db1ef56fb4ecace0a47cc44705dda20f10c0bd64f509fea7cd.0ef7cb179a37032843c28b2c55b8ba7d312d6e44596aa433a59a40fc8abb8f72
loading weights file https://huggingface.co/fassahat/bert-for-patent/resolve/main/pytorch_model.bin from cache at C:\Users\ranax/.cache\huggingface\transformers\4a9afff6de7545db1ef56fb4ecace0a47cc44705dda20f10c0bd64f509fea7cd.0ef7cb179a37032843c28b2c55b8ba7d312d6e44596aa433a59a40fc8abb8f72
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at fassahat/bert-for-patent.
If your task is sim

storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/tokenizer_config.json in cache at C:\Users\ranax/.cache\huggingface\transformers\ea3b3138af3bb2c22a1be9161920935f5b78230ff4a283fc6730acfd35a234ba.b527309dec4079e835b6e29ac399f3cd3ab700485e68976a484dac3ce6ae3911
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\ea3b3138af3bb2c22a1be9161920935f5b78230ff4a283fc6730acfd35a234ba.b527309dec4079e835b6e29ac399f3cd3ab700485e68976a484dac3ce6ae3911
https://huggingface.co/fassahat/bert-for-patent/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to C:\Users\ranax\.cache\huggingface\transformers\tmpwswdihet


storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/vocab.txt in cache at C:\Users\ranax/.cache\huggingface\transformers\37941dec739f471f7ccf28be895d475747ea413a9546daade523fb634dadd4dd.cd71be873e111a6c22b39ab6bcbd9396a088a052b409986b5f2e5c913a024d6e
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\37941dec739f471f7ccf28be895d475747ea413a9546daade523fb634dadd4dd.cd71be873e111a6c22b39ab6bcbd9396a088a052b409986b5f2e5c913a024d6e
https://huggingface.co/fassahat/bert-for-patent/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to C:\Users\ranax\.cache\huggingface\transformers\tmplt_w21k2


storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/tokenizer.json in cache at C:\Users\ranax/.cache\huggingface\transformers\703db49331ef6263cb6756592822649b06687da0232198b3eed7160f3f303d2b.f94e214638d9f00caff260dbbac9bbaf1641ea9e8968cfa305d4b360493b2111
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\703db49331ef6263cb6756592822649b06687da0232198b3eed7160f3f303d2b.f94e214638d9f00caff260dbbac9bbaf1641ea9e8968cfa305d4b360493b2111
https://huggingface.co/fassahat/bert-for-patent/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to C:\Users\ranax\.cache\huggingface\transformers\tmpncyicvpz


storing https://huggingface.co/fassahat/bert-for-patent/resolve/main/special_tokens_map.json in cache at C:\Users\ranax/.cache\huggingface\transformers\37bd2a4ae533dc19cc7e45201c3175c1061a410bf7e0839358641e32d0c4ea8f.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\37bd2a4ae533dc19cc7e45201c3175c1061a410bf7e0839358641e32d0c4ea8f.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/fassahat/bert-for-patent/resolve/main/vocab.txt from cache at C:\Users\ranax/.cache\huggingface\transformers\37941dec739f471f7ccf28be895d475747ea413a9546daade523fb634dadd4dd.cd71be873e111a6c22b39ab6bcbd9396a088a052b409986b5f2e5c913a024d6e
loading file https://huggingface.co/fassahat/bert-for-patent/resolve/main/tokenizer.json from cache at C:\Users\ranax/.cache\huggingface\transformers\703db49331ef6263cb6756592822649b06687da0232198b3eed7160f3f303d2b.f94e214638d9f00caff260db

[{'label': 'LABEL_0', 'score': 0.38417473435401917}]